## Some important notes: 
This challenge is more about data processing, and machine learning techniques. The prepared data could be used to train machine learning or AI models to determine whether a parasite has cancer based on some "factors/predictors". I had a strong background in Machine learning and AI, a few years ago. When I had my undergraduate degree, in statiscis and comoputer science, my focus was on AI and machine learning. After the graduation and my master degree, I shifted my focus more to web development, working with cloud, frontend technologies and backend technologies, such as react, angular, node, python, fastapi, express, java spring boot, docker, microservice, etc. So I am no longer proficient in working with data processing and train models. But I did this question as best as I could. 

Therefore, this kind of challenge is no longer my strength at the moment of this challenge. But I am willing to do some review and refresh my memory if I could be considered as candidates for this type of role. Thank you

### Question 1:
Image by microscope: One representation is that I can use a bitmap with run length encoding. I initially intended to represent images as 2-d array in python, and I can utilize the GPU's parallel processing in pytorch for further analysis easily. But I notice that the question's requirement is to use as less storage/memory as possible. Therefore, a bitmap with run length encoding would more closely align with the requrements. 

Since the large amount of consecutive bits are of the same color(white or black), we will use 33 bits to store each run. For example, the 32 bits stores the length of the run, and the last bit stores the color (0 for white, 1 for black).

In the worst case, there are 10 billion runs(color alternates). Then there would be 1000000000 * 33 bits. This would be worse than using two d array. But only by a constant factor which is 33. In big O notation, the storage is O(n), suppose n is the number of bits in two d array, which is linear in the worse case for both 2d array representation, and bitmap representation. To translate to bytes, you will simply divde that number by 4

But in practice, this representation would be better. Since the situation is likely that in a single row of the 2d array representation, there are only constant number alternations of colors(because the blog is grouped together), then the storage becomes
O(log(n)), which would be better than the two d array

I would represent dye image in the same way, because given the situation of this challenge, the dye image is also binary(whether the dye is lit or not). Bitmap is great for binary representations. 

I also reserached if there are intensities of the dye involved, we will use other representations such as tree

### Qestion 2

In [5]:
# %pip install numpy opencv-python matplotlib
import numpy as np
import cv2
import matplotlib.pyplot as plt

def generate_parasite_image(width, height, min_blob_size=0.25):
    """
    Generate a simulated microscope image with binary blobs representing parasites.
    """
    image = np.zeros((height, width), dtype=np.uint8)
    num_blobs = np.random.randint(1, 5)  # Randomly choose the number of blobs
    
    for _ in range(num_blobs):
        # Randomly choose the center and size of the blob
        center = (np.random.randint(width), np.random.randint(height))
        radius = np.random.randint(int(min_blob_size * min(width, height)), int(0.5 * min(width, height)))
        cv2.circle(image, center, radius, 1, -1)  # Draw the blob
    
    return image

def generate_dye_image(parasite_image, dye_leakage_prob=0.01):
    """
    Generate a simulated dye sensor image based on the given parasite image.
    """
    height, width = parasite_image.shape
    dye_image = np.zeros((height, width), dtype=np.uint8)
    
    # Add dye within the parasites
    dye_image[parasite_image == 1] = 1
    
    # Simulate dye leakage
    leakage = np.random.rand(height, width) < dye_leakage_prob
    dye_image[leakage] = 1
    
    return dye_image

def create_bitmap(image_array):
    """
    Create a bitmap from a binary image array.
    """
    binary_image = (image_array > 0).astype(np.uint8)
    bit_packed_image = np.packbits(binary_image)
    return bit_packed_image

def decode_bitmap(bit_packed_image, width, height):
    """
    Decode a bitmap back to a binary image array.
    """
    binary_image = np.unpackbits(bit_packed_image)
    image_array = binary_image[:width * height].reshape((height, width))
    return image_array

# Parameters
width, height = 1000, 1000 # I can only run for 1000. I will run out of memory if I do 100000

# Generate simulated images
parasite_image = generate_parasite_image(width, height)
dye_image = generate_dye_image(parasite_image)

# Convert to bitmap
parasite_bitmap = create_bitmap(parasite_image)
dye_bitmap = create_bitmap(dye_image)

# Decode bitmap to verify
decoded_parasite_image = decode_bitmap(parasite_bitmap, width, height)
decoded_dye_image = decode_bitmap(dye_bitmap, width, height)


print(f"Parasite Bitmap Size: {len(parasite_bitmap)} bytes")
print(f"Dye Bitmap Size: {len(dye_bitmap)} bytes")


Parasite Bitmap Size: 125000 bytes
Dye Bitmap Size: 125000 bytes


### Question 3

In [4]:
def has_cancer(parasite_bitmap, dye_bitmap, width, height, cancer_threshold=0.1):
    """
    Determine if a parasite has cancer based on the amount of dye present.
    Args:
    - parasite_bitmap: Bit-packed binary image of the parasite.
    - dye_bitmap: Bit-packed binary image of the dye presence.
    - width: Width of the image.
    - height: Height of the image.
    - cancer_threshold: Threshold ratio of dye area to parasite area to classify as cancer.
    
    Returns:
    - bool: True if the parasite has cancer, False otherwise.
    """
    # Decode the bitmaps back to binary images for processing
    parasite_image = decode_bitmap(parasite_bitmap, width, height)
    dye_image = decode_bitmap(dye_bitmap, width, height)
    
    # Calculate the total area of the parasite
    parasite_area = np.sum(parasite_image)
    
    # Calculate the area of dye within the parasite
    dye_within_parasite = np.sum((parasite_image == 1) & (dye_image == 1))
    
    # Determine if the dye area exceeds the threshold
    if dye_within_parasite / parasite_area > cancer_threshold:
        return True
    else:
        return False

cancer_status = has_cancer(parasite_bitmap, dye_bitmap, width, height)
print(f"Parasite has cancer: {cancer_status}")

Parasite has cancer: True


### Quesiton 4

In [6]:
# Qestion 4
def count_bits(bit_packed_image):
    return np.sum(np.unpackbits(bit_packed_image))

def decode_bitmap_subset(bit_packed_image, start, length):
    """
    Decode a subset of a bitmap back to a binary image array.
    """
    binary_image = np.unpackbits(bit_packed_image[start:start + length])
    return binary_image

def has_cancer_new(parasite_bitmap, dye_bitmap, width, height, cancer_threshold=0.1):
    """
    Determine if a parasite has cancer based on the amount of dye present.
    Args:
    - parasite_bitmap: Bit-packed binary image of the parasite.
    - dye_bitmap: Bit-packed binary image of the dye presence.
    - width: Width of the image.
    - height: Height of the image.
    - cancer_threshold: Threshold ratio of dye area to parasite area to classify as cancer.
    
    Returns:
    - bool: True if the parasite has cancer, False otherwise.
    """
    # Calculate the total area of the parasite by counting set bits
    parasite_area = count_bits(parasite_bitmap)
    
    # Efficiently process the bitmaps in chunks to minimize memory usage
    dye_within_parasite = 0
    chunk_size = 1024  # Adjust this size based on available memory and performance
    
    num_chunks = len(parasite_bitmap) // chunk_size + (1 if len(parasite_bitmap) % chunk_size else 0)
    for i in range(num_chunks):
        start = i * chunk_size
        length = min(chunk_size, len(parasite_bitmap) - start)
        parasite_chunk = decode_bitmap_subset(parasite_bitmap, start, length)
        dye_chunk = decode_bitmap_subset(dye_bitmap, start, length)
        
        dye_within_parasite += np.sum(parasite_chunk & dye_chunk)
    
    # Determine if the dye area exceeds the threshold
    return dye_within_parasite / parasite_area > cancer_threshold

cancer_status = has_cancer_new(parasite_bitmap, dye_bitmap, width, height)
print(f"Parasite has cancer: {cancer_status}")

Parasite has cancer: True


We can avoid decoding the entire images by using chunks. We decode images by chunks and process by chunks, so when the threshold might be reached in the middle of the loop, then we avoid processing the rest, improving the speed. It also helps us save memory. 

I am running this on my local computer. Normally, I would run it in cloud env such as google colab, which can utilize GPU for extrememly fast processing. 

### Question 5

I could not come up with effective compression for this question, so I used both claud ai and chatgpt for an answer

Based on my inqueries, Quadtree compression could be an answer. 

My understanding is that we represent the image in tree structure. Each node represent a group of bits with the value. We divide the square image into four quads, and keep diving until all bits in a quads are of the same value. Then such quad becomes a leaf. This could be an effective representation of the images besides the bitmap. 

But given the setting of this challenge, I belive quadtree does not necessarily produce a better run time/storage saving. 

I am using a laptop, and I got kernel error when I tried to generate images of 100000*100000. I believe the memory runs out. I believe it could be feasible to run the code in the cloud. I had experience using google colab before. When I trained machine learning model before and turned large neural network, I typically run the cloud using google colab and run on the GPU. When budget is sufficent, upgrading the hardware might be the most effective approach than dealing with compression techniques. 

### Question 6:

Tools I used for this challenge: 

Google(searching)
Chatgpt(explaining the data structure, such as bitmap)

I am good at writing prompts to the LLM, and ask for answers to my questions in my daily development task, and this can greatly increase my efficiency.

I used chatgpt for the API of the libraries, such as ny functions, and some nuances.

